In [ ]:
# импотируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import History
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
# выводим общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46382 entries, 0 to 46381
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   request_ts       46382 non-null  int64  
 1   user_id          46382 non-null  object 
 2   referer          46382 non-null  object 
 3   geo_id           46381 non-null  float64
 4   component0       46381 non-null  float64
 5   component1       46381 non-null  float64
 6   component2       46381 non-null  float64
 7   component3       46381 non-null  float64
 8   component4       46381 non-null  float64
 9   component5       46381 non-null  float64
 10  component6       46381 non-null  float64
 11  component7       46381 non-null  float64
 12  component8       46381 non-null  float64
 13  component9       46381 non-null  float64
 14  country_id       46381 non-null  object 
 15  region_id        46381 non-null  object 
 16  timezone_id      46381 non-null  object 
 17  browser     

In [ ]:
# удаляем пустые строки
df = df.dropna()

In [ ]:
df.head()

,request_ts,user_id,referer,geo_id,component0,component1,component2,component3,component4,component5,...,component8,component9,country_id,region_id,timezone_id,browser,browser_version,os,os_version,target
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,https://9b48ee5/,4799.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,470e75,f6155e,Chrome Mobile,119.0.0,Android,10,0.0
1,1700986581,46a5f128fd569c764a92c2eaa788095e,https://9b48ee5/,8257.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,44520b,e56e80,Chrome Mobile,111.0.0,Android,10,0.0
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,https://9634fd0/1409e548,3150.0,12498.0,2451.0,10304.0,-6380.0,11608.0,3106.0,...,3347.0,21870.0,c31b4e,616bb9,af47f1,Yandex Browser,20.12.5,Android,11,0.0
3,1700992803,af735816ca19115431ae3d89518c8c91,https://9b48ee5/,2740.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,3c9dca,e56e80,Chrome Mobile,119.0.0,Android,10,0.0
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,https://9b48ee5/,4863.0,11731.0,4045.0,22213.0,-1184.0,-8992.0,9381.0,...,-899.0,16817.0,c31b4e,776e76,10b7947,Yandex Browser,18.11.1,Android,4.4.4,0.0


Из данных видим, что есть очевидно лишний столбец user_id, который не поможет при обучении, данные из столбца geo_id требует перевода в тип object, также приведем к целочисленному типу target, приведем к строковому типу данных referer для последующей обработки

In [ ]:
df.drop(columns=['user_id'], inplace=True)
df['geo_id'] = df["geo_id"].astype("object")
df['target'] = df["target"].astype("int64")
df = df[df['referer'].apply(lambda x: isinstance(x, str))]

Разделим данные из колонки referer на domain и path - чтобы отдельно обрабатывать информацию об основной странице и отдельно - о вкладках, к которые переходил пользователь. Таким образом, удобнее обрабатывать данные о том, переходил ли пользователь куда-то кроме главной страницы. После со здания двух новых колонок удаляем referer

In [ ]:
def split_referer(referer):
    referer = referer[8::]
    parts = referer.split('/', 2)
    domain = parts[0]
    if len(parts) > 1 and parts [1]:
        path = parts[1]
    else:
        path = 'nopath'
    return domain, path

In [ ]:
df[['domain', 'path']] = df['referer'].apply(split_referer).tolist()

In [ ]:
df.drop(columns=['referer'], inplace=True)

Поскольку категориальные признаки нужно будет переводить в числовые и более правильным видится метод OneHotEncoding (ввиду отсутствия естественного порядка в данных (за исключением разве что browser_version и os_version, но для этого нужна дополнительная предобработка, так как разные версии относятся еще и к разным видам), то необходимо посмотреть на количество возможных новых столбцов. Как видим из статистики, данные нужно сокращать (в противном случае размер датасета становится около 9 Гб и не позволяет обучить модель на имеющихся ресурсах). Я применил следующее: обрезал номера версий ОС и браузеров до основного номера, по признакам 'domain', 'path', 'region_id', 'geo_id', 'browser_version', 'timezone_id' - редкие категории (количество которых менее 100) и по признаку 'os_version' (если количество менее 30) заменил на искуствнное значение 0

In [ ]:
columns_to_check = ['domain', 'path', 'country_id', 'geo_id', 'region_id', 'timezone_id', 'browser_version', 'os', 'os_version']
unique_counts = df[columns_to_check].agg('nunique')
print(unique_counts)

domain              1839
path               15188
country_id            12
geo_id              1176
region_id            204
timezone_id           55
browser_version      631
os                     7
os_version           146
dtype: int64


In [ ]:
col_for_del = ['browser_version', 'os_version']
for column in col_for_del:
    df[column] = df[column].astype(str).str.split('.').str[0]

In [ ]:
for column in columns_to_check:
    print(f"Статистика по столбцу '{column}':")
    value_counts = df[column].value_counts()
    print(value_counts)
    print("-" * 30)

Статистика по столбцу 'domain':
domain
72879b4    4221
6a81948    4008
8807153    2756
9f1218f    2444
9b08d64    2174
           ... 
b9ef40d       1
71f14c9       1
7e3ac3c       1
a2fe3fd       1
92bbc3e       1
Name: count, Length: 1839, dtype: int64
------------------------------
Статистика по столбцу 'path':
path
nopath      18470
175de82a      230
1458ef49      209
16658dd6      170
13795cfa      151
            ...  
12c400ec        1
166cf3d8        1
12d9980b        1
16584bd1        1
14cfd614        1
Name: count, Length: 15188, dtype: int64
------------------------------
Статистика по столбцу 'country_id':
country_id
c31b4e     40642
121db33     2518
af12ca       845
1234f1d      714
b98648       682
ac5671       546
110628b      133
eba88b       120
e37756       118
103bf7d       41
122be0f       19
ff9306         3
Name: count, dtype: int64
------------------------------
Статистика по столбцу 'geo_id':
geo_id
3663.0    8160
2521.0    3565
4106.0    1454
8816.0    1426
11

In [ ]:
def rare_100(df, columns):
    for col in columns:
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts <= 100].index
        df[col] = df[col].replace(rare_values, 0)
    return df

def rare_os(df, columns):
    for col in columns:
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts <= 30].index
        df[col] = df[col].replace(rare_values, 0)
    return df

In [ ]:
df = rare_100(df, ['domain', 'path', 'region_id', 'geo_id', 'browser_version', 'timezone_id'])
df = rare_os(df, ['os_version'])

<ipython-input-13-d5f553256292>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(rare_values, 0)


In [ ]:
for column in columns_to_check:
    print(f"Статистика по столбцу '{column}':")
    value_counts = df[column].value_counts()
    print(value_counts)
    print("-" * 30)

Статистика по столбцу 'domain':
domain
0          10767
72879b4     4221
6a81948     4008
8807153     2756
9f1218f     2444
9b08d64     2174
b56ea20     1589
65da82a     1045
9ac1e4f     1033
635e50c      916
8d946e9      914
a140f13      860
62faeb4      756
7ec9b29      696
b5380d6      652
a9b8175      592
9c23da0      582
7b8d3ae      554
b6630fd      476
780ce32      454
7e89575      422
7a4c700      418
97cae77      406
b53adc6      397
8c92747      373
ab7b598      318
94dc6b2      310
be458c6      307
b6e7e37      284
711e64b      280
9836ce2      257
b5152d5      251
a2e88d5      235
76cf7f3      233
ac99fe1      230
9b48ee5      230
60d1206      222
9653126      214
68f5ad8      210
91c550a      209
8ccb495      206
710604b      200
b1bf0d2      199
a5346c1      195
a5ed9c5      189
8406d63      182
bd6d5f3      180
abfc768      173
a0d4ca8      165
adc8e3e      160
8624dd2      159
933ff0b      156
7643ae7      126
9e99781      121
b98fd34      121
93ae6b1      118
9fc5c1c  

Для категориальных признаков применяем OneHotEncoding

In [ ]:
features_to_encode = ['domain', 'path', 'country_id', 'region_id', 'os', 'os_version', 'browser', 'timezone_id', 'browser_version']

In [ ]:
df = pd.get_dummies(df, columns=features_to_encode, prefix=features_to_encode)

In [ ]:
def convert_bool_columns_to_int(df):
    for column in df.columns:
        if df[column].dtype == 'bool':
            df[column] = df[column].astype(int)
    return df

df = convert_bool_columns_to_int(df)

Создаем отдельные датафреймы с обучающими данными и с целевым признаком. Разделяем данные на обучающую, валидационную и тестовую выборки, преобразуем их к формату, читаемому моделью TabNet

In [ ]:
df.to_csv('/content/dfdot.csv', sep=';', encoding='utf-8')
df.to_csv('/content/dfnedot.csv', sep=',', encoding='utf-8')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46381 entries, 0 to 46380
Columns: 298 entries, request_ts to browser_version_99
dtypes: float64(11), int64(287)
memory usage: 105.8 MB


In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

In [ ]:
X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values
y_train = y_train.values
y_valid = y_valid.values
y_test = y_test.values

Ниже приведены примеры обучения модели с разными гиперпараметрами - как параметрами модели, так и параметрами метода fit. Эксперименты показали, что наилучший результат достигнут при параметре batch_size=512, изменение иных параметров относительно дефолтных не улучшило результаты

In [ ]:
clf = TabNetClassifier()
clf.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  eval_metric=['accuracy'],
  max_epochs=100,
  patience=15,
  callbacks=[]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.71081 | val_0_accuracy: 0.50305 |  0:00:14s
epoch 1  | loss: 0.63362 | val_0_accuracy: 0.64714 |  0:00:27s
epoch 2  | loss: 0.60627 | val_0_accuracy: 0.61265 |  0:00:34s
epoch 3  | loss: 0.58788 | val_0_accuracy: 0.66104 |  0:00:41s
epoch 4  | loss: 0.56694 | val_0_accuracy: 0.66439 |  0:00:47s
epoch 5  | loss: 0.55391 | val_0_accuracy: 0.66751 |  0:00:54s
epoch 6  | loss: 0.54771 | val_0_accuracy: 0.68799 |  0:01:00s
epoch 7  | loss: 0.54132 | val_0_accuracy: 0.69829 |  0:01:07s
epoch 8  | loss: 0.53296 | val_0_accuracy: 0.69781 |  0:01:13s
epoch 9  | loss: 0.5288  | val_0_accuracy: 0.69793 |  0:01:19s
epoch 10 | loss: 0.52574 | val_0_accuracy: 0.69877 |  0:01:25s
epoch 11 | loss: 0.52173 | val_0_accuracy: 0.682   |  0:01:32s
epoch 12 | loss: 0.51807 | val_0_accuracy: 0.69517 |  0:01:38s
epoch 13 | loss: 0.5134  | val_0_accuracy: 0.69386 |  0:01:44s
epoch 14 | loss: 0.50902 | val_0_accuracy: 0.69721 |  0:01:51s
epoch 15 | loss: 0.50586 | val_0_accuracy: 0.62606 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
clf2 = TabNetClassifier()
clf.fit(
  X_train, y_train,
  batch_size=2048,
  eval_set=[(X_valid, y_valid)],
  eval_metric=['accuracy'],
  max_epochs=100,
  patience=15,
  callbacks=[]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.73166 | val_0_accuracy: 0.53527 |  0:00:06s
epoch 1  | loss: 0.68274 | val_0_accuracy: 0.59899 |  0:00:11s
epoch 2  | loss: 0.63316 | val_0_accuracy: 0.61037 |  0:00:18s
epoch 3  | loss: 0.60203 | val_0_accuracy: 0.63133 |  0:00:23s
epoch 4  | loss: 0.58394 | val_0_accuracy: 0.62894 |  0:00:29s
epoch 5  | loss: 0.57772 | val_0_accuracy: 0.64966 |  0:00:35s
epoch 6  | loss: 0.57366 | val_0_accuracy: 0.63421 |  0:00:40s
epoch 7  | loss: 0.57098 | val_0_accuracy: 0.6372  |  0:00:46s
epoch 8  | loss: 0.56974 | val_0_accuracy: 0.67649 |  0:00:52s
epoch 9  | loss: 0.56543 | val_0_accuracy: 0.69541 |  0:00:58s
epoch 10 | loss: 0.56568 | val_0_accuracy: 0.70296 |  0:01:03s
epoch 11 | loss: 0.56769 | val_0_accuracy: 0.70272 |  0:01:09s
epoch 12 | loss: 0.56471 | val_0_accuracy: 0.70164 |  0:01:15s
epoch 13 | loss: 0.56396 | val_0_accuracy: 0.70751 |  0:01:21s
epoch 14 | loss: 0.56235 | val_0_accuracy: 0.69984 |  0:01:27s
epoch 15 | loss: 0.56038 | val_0_accuracy: 0.70056 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
clf2 = TabNetClassifier()
clf.fit(
  X_train, y_train,
  batch_size=512,
  eval_set=[(X_valid, y_valid)],
  eval_metric=['accuracy'],
  max_epochs=100,
  patience=15,
  callbacks=[]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69523 | val_0_accuracy: 0.64175 |  0:00:07s
epoch 1  | loss: 0.62519 | val_0_accuracy: 0.66068 |  0:00:14s
epoch 2  | loss: 0.61458 | val_0_accuracy: 0.66008 |  0:00:22s
epoch 3  | loss: 0.61173 | val_0_accuracy: 0.65625 |  0:00:29s
epoch 4  | loss: 0.60522 | val_0_accuracy: 0.64499 |  0:00:37s
epoch 5  | loss: 0.59104 | val_0_accuracy: 0.64678 |  0:00:44s
epoch 6  | loss: 0.56001 | val_0_accuracy: 0.64199 |  0:00:52s
epoch 7  | loss: 0.54527 | val_0_accuracy: 0.64571 |  0:00:58s
epoch 8  | loss: 0.53437 | val_0_accuracy: 0.65325 |  0:01:06s
epoch 9  | loss: 0.52489 | val_0_accuracy: 0.63073 |  0:01:13s
epoch 10 | loss: 0.51193 | val_0_accuracy: 0.62295 |  0:01:20s
epoch 11 | loss: 0.50767 | val_0_accuracy: 0.62331 |  0:01:27s
epoch 12 | loss: 0.50522 | val_0_accuracy: 0.67433 |  0:01:36s
epoch 13 | loss: 0.49941 | val_0_accuracy: 0.72224 |  0:01:44s
epoch 14 | loss: 0.4964  | val_0_accuracy: 0.73889 |  0:01:51s
epoch 15 | loss: 0.49108 | val_0_accuracy: 0.74823 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
clf2 = TabNetClassifier(
    n_d=16,
    n_a=16,
    n_steps=5,
    optimizer_params={'lr': 0.01}
)
clf.fit(
  X_train, y_train,
  batch_size=512,
  eval_set=[(X_valid, y_valid)],
  eval_metric=['accuracy'],
  max_epochs=100,
  patience=15,
  callbacks=[]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69523 | val_0_accuracy: 0.64175 |  0:00:06s
epoch 1  | loss: 0.62519 | val_0_accuracy: 0.66068 |  0:00:14s
epoch 2  | loss: 0.61458 | val_0_accuracy: 0.66008 |  0:00:21s
epoch 3  | loss: 0.61173 | val_0_accuracy: 0.65625 |  0:00:28s
epoch 4  | loss: 0.60522 | val_0_accuracy: 0.64499 |  0:00:35s
epoch 5  | loss: 0.59104 | val_0_accuracy: 0.64678 |  0:00:42s
epoch 6  | loss: 0.56001 | val_0_accuracy: 0.64199 |  0:00:49s
epoch 7  | loss: 0.54527 | val_0_accuracy: 0.64571 |  0:00:57s
epoch 8  | loss: 0.53437 | val_0_accuracy: 0.65325 |  0:01:04s
epoch 9  | loss: 0.52489 | val_0_accuracy: 0.63073 |  0:01:11s
epoch 10 | loss: 0.51193 | val_0_accuracy: 0.62295 |  0:01:18s
epoch 11 | loss: 0.50767 | val_0_accuracy: 0.62331 |  0:01:25s
epoch 12 | loss: 0.50522 | val_0_accuracy: 0.67433 |  0:01:33s
epoch 13 | loss: 0.49941 | val_0_accuracy: 0.72224 |  0:01:41s
epoch 14 | loss: 0.4964  | val_0_accuracy: 0.73889 |  0:01:48s
epoch 15 | loss: 0.49108 | val_0_accuracy: 0.74823 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
